# Part 8 পুনর্বার প্রোটোকল পরিচয় (bis - Introduction to Protocols)

### প্রসঙ্গ (Context) 

এখন আমরা পরিকল্পনার মধ্য দিয়ে চলেছি, আমরা প্রোটোকল নামে একটি নতুন অবজেক্ট প্রবর্তন করব। একটি প্রোটোকল পরিকল্পনাগুলির একটি ক্রম সমন্বয় করে, সেগুলি দূরবর্তী কর্মীদের উপর স্থাপন করে
এবং তাদের একক পাস চালাতে।

এটি একটি উচ্চ স্তরের অবজেক্ট যা একটি জটিল গণনার যুক্তি যুক্ত করে
বিভিন্ন শ্রমিক জুড়ে বিতরণ। প্রোটোকলের প্রধান বৈশিষ্ট্য হ'ল
কর্মীদের মধ্যে এবং শেষ পর্যন্ত প্রেরণ / অনুসন্ধান / ফেরত পাঠানোর ক্ষমতা
চিহ্নিত কর্মীদের মোতায়েন। সুতরাং কোনও ব্যবহারকারী একটি প্রোটোকল ডিজাইন করতে পারেন, এটি আপলোড করতে পারেন
কোনও মেঘকর্মীর কাছে এবং অন্য যে কোনও শ্রমিকরা এটি অনুসন্ধান করতে সক্ষম হবেন,
এটি ডাউনলোড করুন, এবং এটি কর্মীদের উপর থাকা গণনা প্রোগ্রামটি প্রয়োগ করুন
যে এটি সংযুক্ত আছে।

আসুন দেখি কীভাবে এটি ব্যবহার করবেন।

Authors:
- Théo Ryffel - Twitter [@theoryffel](https://twitter.com/theoryffel) - GitHub: [@LaRiffle](https://github.com/LaRiffle)

অনুবাদক:

- Sourav Das - Twitter: [@adventuroussrv](https://twitter.com/adventuroussrv)

### 1. তৈরি করুন এবং মোতায়েন করুন (Create and deploy)

প্রোটোকল জোড়াগুলির তালিকা সরবরাহ করে তৈরি করা হয় `(worker, plan)`. `worker` হয় একটি বাস্তব হতে পারে
কর্মী বা একটি কর্মী আইডি বা একটি কল্পিত কর্মীর প্রতিনিধিত্ব করার জন্য একটি স্ট্রিং। এই
দুটি ক্ষেত্রে পরিকল্পনা হওয়া উচিত তা নির্দিষ্ট করতে শেষ কেসটি তৈরিতে ব্যবহার করা যেতে পারে
স্থাপনার সময়ে একই শ্রমিকের মালিকানাধীন (বা মালিকানাধীন নয়)। `plan` হয় পরিকল্পনা বা একটি পয়েন্টারপ্ল্যান হতে পারে।

In [ ]:
import torch as th
import syft as sy
hook = sy.TorchHook(th)

# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False

আসুন 3 টি পরিকল্পনা সংজ্ঞায়িত করুন এবং এগুলি একটি প্রোটোকলে ফিড করুন। তারা সবাই একটি ইনক্রিমেন্ট অপারেশন করে।

In [ ]:
@sy.func2plan(args_shape=[(1,)])
def inc1(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc2(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc3(x):
    return x + 1

protocol = sy.Protocol([("worker1", inc1), ("worker2", inc2), ("worker3", inc3)])

এখন আমাদের কর্মীদের কাছে প্রোটোকলটি বাঁধতে হবে, যা কল করে করা হয় `.deploy(*workers)`. আসুন কিছু কর্মী তৈরি করা যাক।

In [ ]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")

In [ ]:
workers = alice, bob, charlie

protocol.deploy(*workers)

আপনি দেখতে পাচ্ছেন যে পরিকল্পনাগুলি ইতিমধ্যে উপযুক্ত কর্মীদের কাছে প্রেরণ করা হয়েছে: এটি মোতায়েন করা হয়েছে!

এটি ২ টি পর্যায়ক্রমে করা হয়েছে: প্রথমে আমরা সৃষ্টিতে প্রদত্ত কল্পনাপ্রসূত শ্রমিকদের ম্যাপ করি
সরবরাহকারী কর্মীদের কাছে (স্ট্রিং দ্বারা নামযুক্ত), এবং দ্বিতীয়টি, আমরা সংশ্লিষ্টটি প্রেরণ করি
তাদের প্রতিটি পরিকল্পনা।

### 2. একটি প্রোটোকল চালান (Run a protocol)

প্রোটোকল চালানো মানে সমস্ত পরিকল্পনা ক্রমান্বয়ে সম্পাদন করা। এটি করতে, আপনি কিছু ইনপুট ডেটা সরবরাহ করেন যা প্রথম পরিকল্পনার স্থানে প্রেরণ করা হয়। এই প্রথম পরিকল্পনা
রান করুন এবং এর আউটপুট দ্বিতীয় পরিকল্পনার স্থানে সরানো হয়েছে, ইত্যাদি। চূড়ান্ত
সমস্ত পরিকল্পনা চলার পরে ফলাফলটি ফিরে আসে এবং এটি পয়েন্টারগুলির সমন্বয়ে গঠিত
শেষ পরিকল্পনা অবস্থান।

In [ ]:
x = th.tensor([1.0])
ptr = protocol.run(x)
ptr

In [ ]:
ptr.get()

ইনপুট 1.0 টি 3 টি পরিকল্পনার মধ্যে দিয়ে গেছে এবং তাই 3 বার বাড়ানো হয়েছে, এ কারণেই এটি এখন 4.0 এর সমান!

প্রকৃতপক্ষে, আপনি **ডেটাতে কিছু পয়েন্টারে রিমোট** একটি প্রোটোকল চালাতে পারেন:

In [ ]:
james = sy.VirtualWorker(hook, id="james")

In [ ]:
protocol.send(james)

In [ ]:
x = th.tensor([1.0]).send(james)
ptr = protocol.run(x)
ptr

আপনি দেখতে পান ফলাফলটি জেমসের পয়েন্টার

In [ ]:
ptr = ptr.get()
ptr

In [ ]:
ptr = ptr.get()
ptr

### 3. একটি প্রোটোকল অনুসন্ধান করুন (Search for a protocol)

আসল সেটিংসে আপনি একটি রিমোট প্রোটোকল ডাউনলোড করতে, এটি আপনার কর্মীদের উপর স্থাপন এবং এটি আপনার সাথে ডেটা চালাতে চাইতে পারেন

আসুন একটি প্রোটোকল শুরু করুন **যা নিযুক্ত নয়**, এবং এটি একটি দূরবর্তী কর্মীর উপর রাখি

In [ ]:
protocol = sy.Protocol([("worker1", inc1), ("worker2", inc2), ("worker3", inc3)])
protocol.tag('my_protocol')
protocol.send(james)

In [ ]:
me = sy.hook.local_worker # get access to me as a local worker

প্রোটোকলটি খুঁজতে এখন আমরা একটি অনুসন্ধান শুরু করি

In [ ]:
responses = me.request_search(['my_protocol'], location=james)
responses

আপনার প্রোটোকলের একটি পয়েন্টারে অ্যাক্সেস রয়েছে

In [ ]:
ptr_protocol = responses[0]

সাধারণ পয়েন্টারের মতো আপনি এটিকে ফিরে পেতে পারেন

In [ ]:
protocol_back = ptr_protocol.get()
protocol_back

এবং আমরা অংশ 1 এবং 2 এর মতো করতে পারি।

In [ ]:
protocol_back.deploy(alice, bob, charlie)

x = th.tensor([1.0])
ptr = protocol_back.run(x)
ptr.get()

আরও প্রকৃত বিশ্বের উদাহরণগুলি প্রোটোকল সহ আসবে, তবে আপনি ইতিমধ্যে এই নতুন অবজেক্টের দ্বারা খোলার সমস্ত সম্ভাবনা দেখতে পাবেন!

### গিটহাবে পাইসিফ্ট কে স্টার দিন (Star PySyft on GitHub)

আমাদের সম্প্রদায়কে সাহায্য করার সবচেয়ে সহজ উপায় হল রিপোসিটোরি গুলোতে ষ্টার করা
 এটি আমরা যে অসাধারণ সরঞ্জামগুলি তৈরি করছি তার সচেতনতা বাড়াতে সহায়তা করে।
 
- [Star PySyft](https://github.com/OpenMined/PySyft)

### গিটহাবে আমাদের টিউটোরিয়ালগুলি চয়ন করুন (Pick our tutorials on GitHub!)

ফেডারেটেড এবং প্রাইভেসি-প্রিজারভেভিং লার্নিংয়ের চেহারা কেমন হওয়া উচিত এবং আমরা এটির জন্য ইটগুলি কীভাবে তৈরি করছি সে সম্পর্কে আরও ভাল ধারণা পেতে আমরা সত্যিই দুর্দান্ত টিউটোরিয়াল তৈরি করেছি।

- [Checkout the PySyft tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### আমাদের স্ল্যাক যোগ দিন (Join our Slack!)

সর্বশেষতম অগ্রগতিতে আপ টু ডেট রাখার সর্বোত্তম উপায় হ'ল আমাদের সম্প্রদায়ে যোগদান করা!

- [Join slack.openmined.org](http://slack.openmined.org)

### একটি কোড প্রকল্পে যোগদান করুন! (Join a Code Project!)

আমাদের সম্প্রদায়ে অবদান রাখার সর্বোত্তম উপায় হল কোড অবদানকারী হয়ে উঠুন! যে কোনও সময় আপনি পাইসাইফ্ট গিটহাবে ইস্যু পৃষ্ঠাতে যেতে পারেন এবং "প্রকল্পগুলি" এর জন্য ফিল্টার করতে পারেন। এটি আপনাকে শীর্ষ স্তরের সমস্ত টিকিট দেখিয়ে দেবে যে আপনি কোন প্রকল্পগুলিতে যোগদান করতে পারেন তার একটি ওভারভিউ দেয়! আপনি যদি কোনও প্রকল্পে যোগ দিতে না চান তবে আপনি কিছুটা কোডিং করতে চান তবে আপনি "ভাল প্রথম ইস্যু" চিহ্নিত গিটহাব ইস্যুগুলি অনুসন্ধান করে আরও "ওয়ান অফ" মিনি-প্রকল্পগুলির সন্ধান করতে পারেন।

- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### দান করুন (Donate)

আপনার যদি আমাদের কোডবেসে অবদান রাখার সময় না থাকে তবে তবুও সমর্থন দিতে চান, আপনি আমাদের ওপেন কালেক্টিভেরও Backer হয়ে উঠতে পারেন। সমস্ত অনুদান আমাদের ওয়েব হোস্টিং এবং অন্যান্য সম্প্রদায় ব্যয় যেমন হ্যাকাথনস এবং মেটআপগুলির (hackathons and meetups!) দিকে যায়!

- [Donate through OpenMined's Open Collective Page](https://opencollective.com/openmined)